# Model tests

Using this notebook you should be able to run thee final model

Note the commented out line:
```#out = model.predict_segmentation(inp=im[0],out_fname=path_leaf(im[1]))```
if you un comment this line, the model will save the segmentation outputs to file 

In [ ]:
from model import *
import os # File/directory operations
import ntpath # File/directory operations
import matplotlib
import matplotlib.pyplot as plt
import imageio as imio # Reading images
import glob
from collections import Counter
import itertools
import collections

dataImPaths = {'XTrain':os.path.abspath(os.path.join('Data','Image','XTrain')),
             'XTest':os.path.abspath(os.path.join('Data','Image','XTest')),
             'yTrain':os.path.abspath(os.path.join('Data','Image','yTrain')),
             'yTest':os.path.abspath(os.path.join('Data','Image','yTest'))}

In [ ]:
model = vgg_unet(43, 256, 416)

In [ ]:
model.load_weights('Weights/vgg_unet_1.47-CE-Ext2-Fixed-N.h5')

In [ ]:
imXTrain = glob.glob(os.path.join(dataImPaths['XTrain'], "*"))
imXTest = glob.glob(os.path.join(dataImPaths['XTest'], "*"))

imYTrain = glob.glob(os.path.join(dataImPaths['yTrain'], "*"))
imYTest = glob.glob(os.path.join(dataImPaths['yTest'], "*"))

filePairPathsTrain = list(zip(imXTrain, imYTrain))
filePairPathsTest = list(zip(imXTest, imYTest))

In [ ]:
def getUniqueCountFormImage(imPath):
    img = np.unique(np.array(imPath).astype(np.uint16))
    return dict(zip(img,itertools.repeat(1)))

def addDicts(d1, d2):
    A = Counter(d1)
    B = Counter(d2)
    return dict(A + B)

def path_leaf(path):
    """This function gets the file name from a path"""
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def getClassFromID(id):
    data = pd.read_csv('Data/classes.csv')
    return data.loc[data['_id'] == id, '_name'].values[0]

In [ ]:
testPairs = {}
for im in filePairPathsTest:
    out = model.predict_segmentation(inp=im[0],out_fname=path_leaf(im[1]))
    #out = predict(model, im[0])
    true = imio.imread(im[1])
                        
    true = np.array(true).ravel()
    out = np.array(out).ravel()
                        
    t = 0.001
    uniqueTrue = reduceSSCNoise(true,0)
    uniquePred = reduceSSCNoise(out,t)
    
    
    uniqueTrue.sort()
    uniquePred.sort()
    
    testPairs[path_leaf(im[0])] = {
        'yTest':uniqueTrue,
        'yPred':uniquePred
    }
    
    #print(path_leaf(im[1]))
    print([getClassFromID(x) for x in testPairs[path_leaf(im[0])]['yTest']])
    #print(path_leaf(im[0]))
    print([getClassFromID(x) for x in testPairs[path_leaf(im[0])]['yPred']])
    print(SSC(true,reduceSSCNoise(out,t)))
    #print('\n')

In [ ]:
testPairs = {}
for im, real in zip(filePairPathsTrain, imXTrain):
    #out = model.predict_segmentation(inp=im[0],out_fname=path_leaf(im[1]))
    out = model.predict_segmentation(inp=im[0])
    true = imio.imread(im[1])
    
    uniqueTrue = reduceSSCNoise(true,0)
    uniquePred = reduceSSCNoise(out)
    
    
    uniqueTrue.sort()
    uniquePred.sort()
    
    testPairs[path_leaf(im[0])] = {
        'yTest':uniqueTrue,
        'yPred':uniquePred
    }
    
    print(path_leaf(im[1]))
    plt.imshow(imio.imread(real))
    plt.show()
    print([getClassFromID(x) for x in testPairs[path_leaf(im[0])]['yTest']])
    #print(path_leaf(im[0]))
    print([getClassFromID(x) for x in testPairs[path_leaf(im[0])]['yPred']])
    #print(SSC(true,reduceSSCNoise(out)))
    #print('\n')

In [ ]:
getClassFromID(1)

In [ ]:
imYTrain = glob.glob(os.path.join(dataImPaths['yTrain'], "*"))
imYTest = glob.glob(os.path.join(dataImPaths['yTest'], "*"))

In [ ]:
trainDist = {}
for image in imYTrain:
    imDist = getUniqueCountFormImage(imio.imread(image))
    print(imDist)
    trainDist = addDicts(trainDist, imDist)
    
trainDist = dict(collections.OrderedDict(sorted(trainDist.items())))

testDist = {}
for image in imYTest:
    imDist = getUniqueCountFormImage(imio.imread(image))
    testDist = addDicts(testDist, imDist)
    
testDist = dict(collections.OrderedDict(sorted(testDist.items())))


In [ ]:
matplotlib.rcParams['figure.figsize'] = [14, 5]

df = pd.DataFrame.from_dict(trainDist, orient='index').astype(float)
df1 = pd.DataFrame.from_dict(testDist, orient='index').astype(float)

plt.bar(list(df.index),list(df.iloc[:,0]),alpha = 0.5, label='Train')
plt.bar(list(df1.index),list(df1.iloc[:,0]),alpha = 0.5, label='Test')
plt.xticks(range(0,43))
# x-axis label 
plt.xlabel('class') 
# y-axis label 
plt.ylabel('# of occurrences') 
# Legend
plt.legend(title="Dataset",loc=1)
# show the plot
plt.show()